# Exploring DNN learning with TensorFlow

In this assignment we'll dive a little deeper with a series of hands on exercises to better understand DNN learning with Tensorflow. Remember that if you are taking the class for a certificate we will be asking you questions about the assignment in the test!

We start by setting up the problem for you.

In [1]:
# Load libraries
import sys

import tensorflow as tf

In [2]:
# This script requires TensorFlow 2 and Python 3.
if sys.version_info.major < 3:
    raise Exception((f"The script is developed and tested for Python 3. "
                     f"Current version: {sys.version_info.major}"))

if tf.__version__.split('.')[0] != '2':
    raise Exception((f"The script is developed and tested for tensorflow 2. "
                     f"Current version: {tf.__version__}"))

In [3]:
# Load in fashion MNIST
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

# Define the base model
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])



4422102/4422102 [==============================] - 0s 0us/step


Neural Networks learn the best when the data is scaled / normalized to fall in a constant range. One practitioners often use is the range [0,1]. How might you do this to the training and test images used here?

*A hint: these images are saved in the standard [RGB](https://www.rapidtables.com/web/color/RGB_Color.html) format*

In [6]:
training_images  = training_images / 255.0

test_images = test_images / 255.0

"""This is because the RGB data format uses values in the range [0,255] to represent each color and so dividing the color values by 255 will result in the data falling in the range [0,1]. Note that we need to make sure to use floating point division or the number will always be either 0 or 1 instead of in the range [0,1]."""

Using these improved images lets compile our model using an adaptive optimizer to learn faster and a categorical loss function to differentiate between the the various classes we are trying to classify. Since this is a very simple dataset we will only train for 5 epochs.

In [7]:
# compile the model
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

# fit the model to the training data
model.fit(training_images, training_labels, epochs=5)

# test the model on the test data
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.8758 - accuracy: 0.7074
Epoch 2/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.5496 - accuracy: 0.8043
Epoch 3/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.4888 - accuracy: 0.8274
Epoch 4/5
1875/1875 [==============================] - 15s 8ms/step - loss: 0.4577 - accuracy: 0.8379
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.4583 - accuracy: 0.8345


[0.45827504992485046, 0.8345000147819519]

Once it's done training -- you should see an accuracy value at the end of the final epoch. It might look something like 0.8648. This tells you that your neural network is about 86% accurate in classifying the training data. I.E., it figured out a pattern match between the image and the labels that worked 86% of the time. But how would it work with unseen data? That's why we have the test images. We can call ```model.evaluate```, and pass in the two sets, and it will report back the loss for each. This should reach about .8747 or thereabouts, showing about 87% accuracy. Not Bad!

But what did it actually learn? If we inference on the model using ```model.predict``` we get out the following list of values. **What does it represent?**

*A hint: trying running ```print(test_labels[0])```*

In [8]:
classifications = model.predict(test_images)
print(classifications[0])

313/313 [==============================] - 1s 3ms/step
[6.6821877e-07 4.4758433e-08 4.2133479e-06 4.5932243e-06 1.1792221e-05
 1.1232433e-01 1.3825409e-05 3.6249653e-01 4.3186136e-03 5.2082545e-01]


Let's now look at the layers in your model. What happens if you double the number of neurons in the dense layer. What different results do you get for loss, training time etc? Why do you think that's the case?

In [9]:
NUMBER_OF_NEURONS = 1024

# define the new model
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(NUMBER_OF_NEURONS, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

# compile fit and evaluate the model again
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 28s 15ms/step - loss: 0.8028 - accuracy: 0.7297
Epoch 2/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.5152 - accuracy: 0.8170
Epoch 3/5
1875/1875 [==============================] - 23s 12ms/step - loss: 0.4620 - accuracy: 0.8360
Epoch 4/5
1875/1875 [==============================] - 23s 12ms/step - loss: 0.4336 - accuracy: 0.8464
Epoch 5/5
313/313 [==============================] - 2s 6ms/step - loss: 0.4394 - accuracy: 0.8442


[0.43938037753105164, 0.8442000150680542]

Consider the effects of additional layers in the network instead of simply more neurons to the same layer. First update the model to add an additional dense layer into the model between the two existing Dense layers.

In [12]:
YOUR_NEW_LAYER = tf.keras.layers.Dense(512, activation=tf.nn.relu)  # adding 512 to this layer instead of 1024 neuron in the second layer

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    YOUR_NEW_LAYER,
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])

"""When training your new model if you also used a large number of Neurons in your new layer, e.g., 512, you should find that training accuracy improves in a similar manner to doubling the number of Neurons in the single layer. If you are lucky it might even be better! If you chose a small number of Neurons you may have seen no effect. This is simply an artifact of the particular model structure and dataset we are using. Again, remember there is no one size fits all solution in Neural Network model design.'''

Lets then compile, fit, and evaluate our model. What happens to the error? How does this compare to the original model and the model with double the number of neurons?

In [13]:
# compile fit and evaluate the model again
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)
model.evaluate(test_images, test_labels)

Epoch 1/5
1875/1875 [==============================] - 20s 10ms/step - loss: 0.6963 - accuracy: 0.7437
Epoch 2/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.4689 - accuracy: 0.8299
Epoch 3/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.4149 - accuracy: 0.8481
Epoch 4/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.3837 - accuracy: 0.8602
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.4043 - accuracy: 0.8485


[0.4043135643005371, 0.8485000133514404]

Before you trained, you normalized the data. What would be the impact of removing that? To see it for yourself fill in the following lines of code to get a non-normalized set of data and then re-fit and evaluate the model using this data.

In [14]:
# get new non-normalized mnist data
training_images_non = training_images * 255

test_images_non = test_images * 255

# re-compile, re-fit and re-evaluate
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(512, activation=tf.nn.relu),
                                    YOUR_NEW_LAYER,
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(training_images_non, training_labels, epochs=5)
model.evaluate(test_images_non, test_labels)
classifications = model.predict(test_images_non)

Epoch 1/5
1875/1875 [==============================] - 22s 12ms/step - loss: 0.4793 - accuracy: 0.8281
Epoch 2/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.3638 - accuracy: 0.8661
Epoch 3/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.3266 - accuracy: 0.8791
Epoch 4/5
1875/1875 [==============================] - 20s 11ms/step - loss: 0.3026 - accuracy: 0.8877
Epoch 5/5
313/313 [==============================] - 1s 3ms/step


Sometimes if you set the training for too many epochs you may find that training stops improving and you wish you could quit early. Good news, you can! TensorFlow has a function called ```Callbacks``` which can check the results from each epoch. Modify this callback function to make sure it exits training early but not before reaching at least the second epoch!

*A hint: logs.get(METRIC_NAME) will return the value of METRIC_NAME at the current step*

In [15]:
"""Finally we added a custom callback function to help us exit training early if we reached some goal. In this case to exit early but only upon reaching at least the 2nd epoch one thing we could do is exit once the training accuracy is over 86% (as in all of our training runs we got as high as an 83.5% accuracy on the first epoch). One way to do that would be:"""


# define and instantiate your custom Callback
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.86 ):
      self.model.stop_training = True
callbacks = myCallback()

# re-compile, re-fit and re-evaluate
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(28,28)),
                            tf.keras.layers.Dense(512, activation=tf.nn.relu),
                            YOUR_NEW_LAYER,
                            tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
model.compile(optimizer = tf.keras.optimizers.Adam(),
      loss = 'sparse_categorical_crossentropy',
      metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5, callbacks=[callbacks])

Epoch 1/5
1875/1875 [==============================] - 20s 10ms/step - loss: 0.6544 - accuracy: 0.7618
Epoch 2/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.4469 - accuracy: 0.8374
Epoch 3/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.4017 - accuracy: 0.8533
Epoch 4/5
1875/1875 [==============================] - 21s 11ms/step - loss: 0.3727 - accuracy: 0.8629
